Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe.

To create the above dataframe:

The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.
If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [33]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import json 
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim 
import requests 
from pandas.io.json import json_normalize
import folium 
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

Solving environment: done

## Package Plan ##

  environment location: /opt/anaconda3

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-2.0.0                |     pyh9f0ad1d_0          63 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          97 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-2.0.0-pyh9f0ad1d_0



geographiclib-1.50   | 34 KB     | ##################################### | 100% 
geopy-2.0.0          | 63 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done


In [7]:
source=requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup=BeautifulSoup(source, 'xml')
table=soup.find('table', attrs={'class':'wikitable sortable'})

In [3]:
column_names=['Postalcode','Borough','Neighborhood']
df=pd.DataFrame(columns = column_names)

In [8]:
for tr_cell in table.find_all('tr'):
    row=[]
    for td_cell in tr_cell.find_all('td'):
        row.append(td_cell.text.strip())
    if len(row)==3:
        df.loc[len(df)] = row

In [9]:
df.head()

,Postalcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [15]:
df1=df[df['Borough']!='Not assigned'] #Drop the rows where Borough is 'Not assigned'
df1.head()

,Postalcode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [23]:
df2=df1.groupby(['Postalcode','Borough'], sort=False).agg(', '.join) #Combine the neighbourhoods with same Postalcode
df2.reset_index(inplace=True)
#Replace the name of the neighbourhoods 'Not assigned' with names of Borough
df2['Neighborhood']=np.where(df2['Neighborhood']=='Not assigned',df2['Borough'], df2['Neighborhood']) 
df2.head()

,Postalcode,Borough,Neighborhood
0,M3A,North York,"Parkwoods, Parkwoods"
1,M4A,North York,"Victoria Village, Victoria Village"
2,M5A,Downtown Toronto,"Regent Park, Harbourfront, Regent Park, Harbou..."
3,M6A,North York,"Lawrence Manor, Lawrence Heights, Lawrence Man..."
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government, Q..."


In [24]:
df2.shape

(103, 3)

In [25]:
#Import the csv file conatining the latitudes and longitudes for various neighbourhoods in Canada
lat_lon=pd.read_csv('https://cocl.us/Geospatial_data')
lat_lon.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [28]:
#Merge two tables 
lat_lon.rename(columns={'Postal Code':'Postalcode'}, inplace=True)
df3=pd.merge(df2, lat_lon, on='Postalcode')
df3.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,"Parkwoods, Parkwoods",43.753259,-79.329656
1,M4A,North York,"Victoria Village, Victoria Village",43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront, Regent Park, Harbou...",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights, Lawrence Man...",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government, Q...",43.662301,-79.389494


In [30]:
df4=df3[df3['Borough'].str.contains('Toronto',regex=False)]
df4.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Regent Park, Harbourfront, Regent Park, Harbou...",43.654260,-79.360636
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government, Q...",43.662301,-79.389494
9,M5B,Downtown Toronto,"Garden District, Ryerson, Garden District, Rye...",43.657162,-79.378937
15,M5C,Downtown Toronto,"St. James Town, St. James Town",43.651494,-79.375418
19,M4E,East Toronto,"The Beaches, The Beaches",43.676357,-79.293031


In [37]:
map_toronto = folium.Map(location=[43.654260, -79.360636], zoom_start=10)

for lat,lng,borough,neighborhood in zip(df4['Latitude'],df4['Longitude'],df4['Borough'],df4['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
    [lat,lng],
    radius=5,
    popup=label,
    color='blue',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(map_toronto)
map_toronto

Using KMeans clustering for the clsutering of the neighbourhoods in Toronto

In [43]:
#set number of clusters
k=5
toronto_clustering=df4.drop(['Postalcode','Borough','Neighborhood'],1)

#run k-means clustering
kmeans = KMeans(n_clusters=k,random_state=0).fit(toronto_clustering)
kmeans.labels_[0:10]
#df4.insert(0, 'Cluster Labels', kmeans.labels_)
df4.head()

,Cluster Labels,Postalcode,Borough,Neighborhood,Latitude,Longitude
2,0,M5A,Downtown Toronto,"Regent Park, Harbourfront, Regent Park, Harbou...",43.654260,-79.360636
4,0,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government, Q...",43.662301,-79.389494
9,0,M5B,Downtown Toronto,"Garden District, Ryerson, Garden District, Rye...",43.657162,-79.378937
15,0,M5C,Downtown Toronto,"St. James Town, St. James Town",43.651494,-79.375418
19,1,M4E,East Toronto,"The Beaches, The Beaches",43.676357,-79.293031


In [44]:
# create map
map_clusters = folium.Map(location=[43.654260, -79.360636],zoom_start=10)

# set color scheme for the clusters
x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, neighbourhood, cluster in zip(df4['Latitude'], df4['Longitude'], df4['Neighborhood'], df4['Cluster Labels']):
    label = folium.Popup(' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters